### Running this on GCE
Lets make sure it is picking up the GPU

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3024345277074842187
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4932634607361116832
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5445872812009753776
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7382384640
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11240677277970821835
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"
]


In [14]:
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from PIL import Image

In [3]:
train_dir = '../data/frames/train/'
test_dir = '../data/frames/test/'

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

### Tensorflow Inputs
You can create these generators by pointing TF to a directory and it will automatically intentify the targets (based on the sub-folder) and batch the images

In [5]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=32,
                                                   class_mode='binary',
                                                   target_size=(352,240))
test_generator = test_datagen.flow_from_directory(test_dir,
                                                   batch_size=32,
                                                   class_mode='binary',
                                                   target_size=(352,240))

Found 8726 images belonging to 2 classes.
Found 2152 images belonging to 2 classes.


### CNN Layers
Lets make an CNN with 3x convolutions followed by pooling before flattening to our output  
Obviously, there is a lot we can manipulate and test within the NN architecture

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(352,240, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#### Optimizers
RMSProp(0.001) - used based off of example code, but accuracy never moved  
Found https://medium.com/octavian-ai/which-optimizer-and-learning-rate-should-i-use-for-deep-learning-5acb418f9b2  
Adam(0.001) - Also no movement in accuracy. validation accuracy never moved  
Found https://stackoverflow.com/questions/37213388/keras-accuracy-does-not-change  
SGD(0.001) - Movement! 0.71 acc after 20 epochs  
SGD(0.01) - Lower LR = bigger leaps. 0.71 acc after 20 epochs lets call it there  

Obviously, lots of places to optimize here. Use of some grid search and lots of GPU

In [7]:
model.compile(optimizer=SGD(lr=0.01),
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [8]:
history = model.fit_generator(train_generator, 
                             validation_data=test_generator,
                             steps_per_epoch=100,
                             epochs=20,
                             validation_steps=50,
                             verbose=1)

Epoch 1/20
100/100 [==============================] - 42s 416ms/step - loss: 0.6738 - accuracy: 0.5744 - val_loss: 0.6236 - val_accuracy: 0.6981
Epoch 2/20
100/100 [==============================] - 40s 400ms/step - loss: 0.6248 - accuracy: 0.6541 - val_loss: 0.6320 - val_accuracy: 0.6175
Epoch 3/20
100/100 [==============================] - 40s 402ms/step - loss: 0.6099 - accuracy: 0.6712 - val_loss: 0.5868 - val_accuracy: 0.7100
Epoch 4/20
100/100 [==============================] - 29s 291ms/step - loss: 0.5855 - accuracy: 0.6959 - val_loss: 0.5984 - val_accuracy: 0.6637
Epoch 5/20
100/100 [==============================] - 30s 295ms/step - loss: 0.5731 - accuracy: 0.7038 - val_loss: 0.5513 - val_accuracy: 0.7300
Epoch 6/20
100/100 [==============================] - 27s 268ms/step - loss: 0.5579 - accuracy: 0.7253 - val_loss: 0.5637 - val_accuracy: 0.6919
Epoch 7/20
100/100 [==============================] - 26s 262ms/step - loss: 0.5587 - accuracy: 0.7257 - val_loss: 0.5596 - val_ac

In [29]:
model.save('../models/indoor_outdoor_sgd_01.h5')